In [2]:

import pandas as pd
import datetime
import ccxt
from datetime import datetime, timezone, timedelta

import pandas as pd
import pandas_ta as ta
import time 
import json
import numpy as np

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns


exchange='bitmex'
symbol = 'XBTUSDT'
timeframe = '5m'  # 1 day : 1D timeframe
size=1000
limit=5000
takeprofit=20
deviation=1


with open('./file_bitmex.json') as f:
    data = json.load(f)

# Access the values from the loaded JSON data
key_value = data["key"]
secret_value = data["secret"]


# Initialize the exchange
exchange = eval(f'ccxt.{exchange}')({
    'enableRateLimit': True, 
    'options': {
        'adjustForTimeDifference': True,
        'recvWindow': 50000,
        'defaultType': 'swap',    
        'timeDifference': 5000
    },
    'apiKey': key_value,
    'secret': secret_value 
})



# fetching the latest price
def fetch_price(symbol):
    ''' 
    return: float=price

    '''
    price=exchange.fetchTicker(symbol)['info']['lastPrice']
    return float(price)
    

def check_open_orders(symbol):
    '''
    return: list = open orders
    '''
    return exchange.fetchOpenOrders(symbol)



def open_long(symbol,size):
    price=fetch_price(symbol)
    exchange.privatePostOrder({"symbol":f"{symbol}",
                            "ordType":"Limit",
                            "side":"Buy",
                            "timeInForce":"GTC",
                            "quantity":f"{size}",
                            "price":f"{price-deviation}",
                            "execInst":"ParticipateDoNotInitiate"})
    return price
    
    


def open_short(symbol,size):
    price=fetch_price(symbol)
    exchange.privatePostOrder({"symbol":f"{symbol}",
                            "ordType":"Limit",
                            "side":"Sell",
                            "timeInForce":"GTC",
                            "quantity":f"{size}",
                            "price":f"{price+deviation}",
                            "execInst":"ParticipateDoNotInitiate"})
    return price
    

def close_short(symbol,size):
    price=fetch_price(symbol)
    exchange.privatePostOrder({"symbol":f"{symbol}",
                    "ordType":"Limit",
                    "side":"Buy",
                    "timeInForce":"GTC",
                    "quantity":f"{size}",
                    "price":f"{price}",
                    "execInst":"ReduceOnly"})
    


def close_long(symbol,size):
    price=fetch_price(symbol)
    exchange.privatePostOrder({"symbol":f"{symbol}",
                    "ordType":"Limit",
                    "side":"Sell",
                    "timeInForce":"GTC",
                    "quantity":f"{size}",
                    "price":f"{price}",
                    "execInst":"ReduceOnly"})
    
    



def logic_exec(symbol,size,timeframe,price,id,isLong,isShort):


    # Get the OHLCV (Open, High, Low, Close, Volume) data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=None, limit=300)

    # creating dataframe of ohlcv
    df=pd.DataFrame(ohlcv)


    df['ema'] = ta.ema(df[4], window=13)

    ### PSAR
    d=ta.psar(df[2],df[3],df[4],0.06,0.06,0.6)
    d1=ta.ema(df[4][-14:-1],13).iloc[-1]

    #### plotting chart for visualisation ######
    plt.figure(figsize=(20,10))
    sns.lineplot(x=df[0],y=df[4],data=df)
    sns.scatterplot(x=df[0],y=d['PSARs_0.06_0.6'],data=df,color='red')
    sns.scatterplot(x=df[0],y=d['PSARl_0.06_0.6'],data=df,color='green')
    sns.lineplot(x=df[0],y=df['ema'],color='brown')
    plt.savefig(f'./images/plot.png')
    plt.close()
    

##### getting latest value of candle and PSAR ####
    latest_val=d.iloc[-1]



### creating logic  #####
    
    if latest_val['PSARl_0.06_0.6']>0 and latest_val['PSARr_0.06_0.6']==1:
        if id==None and df[4].iloc[-1]>d1:

            while exchange.private_get_position({"symbol":symbol})[0]['isOpen']!=True:
                exchange.private_delete_order_all({"symbol":symbol})
                price=open_long(symbol,size)
                time.sleep(5)
                print('buy')

            
            exchange.privatePostOrder({"symbol":f"{symbol}",
                            "ordType":"Limit",
                            "side":"Sell",
                            "timeInForce":"GTC",
                            "quantity":f"{size}",
                            "price":f"{price+takeprofit}",
                            "execInst":"ReduceOnly"})
            

            id=True
            isLong=True


        elif id!=None and isShort:
            print('Short position close has been placed')

            while(exchange.private_get_position({"symbol":symbol})[0]['isOpen']):
                exchange.private_delete_order_all({"symbol":symbol})
                close_short(symbol,size)
                time.sleep(5)

            isLong=False
            id=None

        


            
    elif latest_val['PSARs_0.06_0.6']>0 and latest_val['PSARr_0.06_0.6']==1:
        if id==None and df[4].iloc[-1]<d1:
            
            while exchange.private_get_position({"symbol":symbol})[0]['isOpen']!=True:
                exchange.private_delete_order_all({"symbol":symbol})
                price=open_short(symbol,size)
                print('sell')
                time.sleep(5)
            
            

            
            exchange.privatePostOrder({"symbol":f"{symbol}",
                            "ordType":"Limit",
                            "side":"Buy",
                            "timeInForce":"GTC",
                            "quantity":f"{size}",
                            "price":f"{price-takeprofit}",
                            "execInst":"ReduceOnly"})
            

            id=True
            isShort=True


        elif id!=None and isLong:
            print('Long position close has been placed')

            while(exchange.private_get_position({"symbol":symbol})[0]['isOpen']):
                close_long(symbol,size)
                time.sleep(5)

                            
            isShort=False
            id=None
        



    time.sleep(15)

    x=exchange.private_get_position({"symbol":symbol})[0]['isOpen']
    if x==False:
        exchange.private_delete_order_all({"symbol":symbol})
        id=None
        isLong=False
        isShort=False

    elif x==True:
        # if isLong:
        #     exchange.privatePostOrder({"symbol":f"{symbol}",
        #                     "ordType":"Limit",
        #                     "side":"Sell",
        #                     "timeInForce":"GTC",
        #                     "quantity":f"{size}",
        #                     "price":f"{price+takeprofit}",
        #                     "execInst":"ReduceOnly"})
            
        # elif isShort:
        #     exchange.privatePostOrder({"symbol":f"{symbol}",
        #                     "ordType":"Limit",
        #                     "side":"Buy",
        #                     "timeInForce":"GTC",
        #                     "quantity":f"{size}",
        #                     "price":f"{price-takeprofit}",
        #                     "execInst":"ReduceOnly"})
    
        time.sleep(300)


    return logic_exec(symbol,size,timeframe,price,id,isLong,isShort)


logic_exec(symbol,size,timeframe,fetch_price(symbol),id=None,isLong=False,isShort=False)



KeyboardInterrupt: 